In [ ]:
from os.path import join
import json
import pandas as pd
from collections import defaultdict, ChainMap

In [ ]:
annotations = {
  'B18': {
    'path': '../../MI/data/MI_annotations_B18_20210903.jsonl'
  },
  'B33': {
    'path': '../../MI/data/MI_annotations_B33_20210903.jsonl'
  }
}
data_dir = '../../MI/data'
submissions_in_path = '../../MI/data/mi_submissions_20210901.csv'
submissions_out_filename = 'mi_submissions_20210903.csv'

In [ ]:
submissions_df = pd.read_csv(submissions_in_path).set_index('id')

In [ ]:
def load_labels(labels_path):
  labeled_submissions = [
    json.loads(line)
    for line in open(labels_path)
  ]
  coi_labels_by_id = defaultdict(set)
  for submission in labeled_submissions:
    labels = set.union(
      *(set(annotation['labels'])
        for annotation in submission['annotations'].values())
    )
    coi_labels_by_id[submission['plan_id']] = labels
  return coi_labels_by_id

In [ ]:
labels = dict(ChainMap(
  *(load_labels(annotation['path'])
    for annotation in annotations.values())
))

In [ ]:
submissions_df['labels'] = submissions_df.index.to_series().apply(lambda _id: list(labels.get(_id, [])))

In [ ]:
submissions_df.to_csv(join(data_dir, submissions_out_filename))